# 1. 어린이보호구역 전처리

### 어린이보호구역 버퍼 생성 (9번데이터 기준, 100m반경, 원모양)
- 어린이 보호구역을 점(좌표)가 아닌 구역으로 인지하기 위함입니다.
- 어린이 보호구역은 시설의 정문을 기준으로 하지만, 현재 데이터는 정문 좌표가 아니므로 버퍼 범위의 오류가 있을 수 있습니다. 

### 어린이보호구역 도로 추출
- 지적도 중 도로를 기준으로 어린이 보호구역과 겹치는 도로를 추출합니다.
- QGIS 프로그램을 사용하였으며, 본 파일에서는 시각화를 통한 확인만 진행했습니다.

In [1]:
#데이터셋 다운로드
from geoband.API import *

GetCompasData('SBJ_2012_001', '1', '1.오산시_주정차단속(2018~2020).csv')
GetCompasData('SBJ_2012_001', '2', '2.오산시_어린이교통사고_격자.geojson')
GetCompasData('SBJ_2012_001', '3', '3.오산시_차량등록현황_격자.geojson')
GetCompasData('SBJ_2012_001', '4', '4.오산시_연령별_거주인구격자(총인구).geojson')
GetCompasData('SBJ_2012_001', '5', '5.오산시_연령별_거주인구격자(유소년).geojson')
GetCompasData('SBJ_2012_001', '6', '6.오산시_연령별_거주인구격자(생산가능인구).geojson')
GetCompasData('SBJ_2012_001', '7', '7.오산시_연령별_거주인구격자(고령).geojson')
GetCompasData('SBJ_2012_001', '8', '8.오산시_유동인구(2019).csv')
GetCompasData('SBJ_2012_001', '9', '9.오산시_어린이보호구역.csv')
GetCompasData('SBJ_2012_001', '10', '10.오산시_학교위치정보.csv')
GetCompasData('SBJ_2012_001', '11', '11.오산시_초등학교_통학구.geojson')
GetCompasData('SBJ_2012_001', '12', '12.오산시_중학교_학군.geojson')
GetCompasData('SBJ_2012_001', '13', '13.오산시_어린이집_유치원현황.csv')
GetCompasData('SBJ_2012_001', '14', '14.오산시_기상데이터(2010~2019).csv')
GetCompasData('SBJ_2012_001', '15', '15.오산시_무인교통단속카메라.csv')
GetCompasData('SBJ_2012_001', '16', '16.오산시_도로안전표지표준데이터.csv')
GetCompasData('SBJ_2012_001', '17', '17.오산시_횡단보도.geojson')
GetCompasData('SBJ_2012_001', '18', '18.오산시_과속방지턱표준데이터.csv')
GetCompasData('SBJ_2012_001', '19', '19.오산시_신호등.geojson')
GetCompasData('SBJ_2012_001', '20', '20.오산시_CCTV설치현황.csv')
GetCompasData('SBJ_2012_001', '21', '21.오산시_인도.geojson')
GetCompasData('SBJ_2012_001', '22', '22.오산시_버스정류장.csv')
GetCompasData('SBJ_2012_001', '23', '23.오산시_상세도로망_LV6.geojson')
GetCompasData('SBJ_2012_001', '24', '24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
GetCompasData('SBJ_2012_001', '25', '25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
GetCompasData('SBJ_2012_001', '26', '26.평일_전일_오산시_혼잡시간강도_Level6.csv')
GetCompasData('SBJ_2012_001', '27', '27.오산시_도로명주소_건물.geojson')
GetCompasData('SBJ_2012_001', '28', '28.오산시_건물연면적_격자.geojson')
GetCompasData('SBJ_2012_001', '29', '29.오산시_체육시설현황.csv')
GetCompasData('SBJ_2012_001', '30', '30.오산시_학원_및_교습소_현황.csv')
GetCompasData('SBJ_2012_001', '31', '31.오산시_법정경계(시군구).geojson')
GetCompasData('SBJ_2012_001', '32', '32.오산시_행정경계(읍면동).geojson')
GetCompasData('SBJ_2012_001', '33', '33.오산시_법정경계(읍면동).geojson')
GetCompasData('SBJ_2012_001', '34', '34.오산시_지적도.geojson')
GetCompasData('SBJ_2012_001', '35', '35.코드정의서.xlsx')

## 어린이보호구역 버퍼생성

In [2]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os

from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [3]:
import warnings 
warnings.filterwarnings(action='ignore')

In [4]:
#어린이 보호구역 좌표 불러오기
data9= pd.read_csv('9.오산시_어린이보호구역.csv')
data9.head()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749


In [5]:
#원활한 공간분석을 위해 어린이보호구역 데이터 좌표를 geometry형식으로 저장합니다.
data9['보호구역_경도'] = data9['보호구역_경도'].astype(float)
data9['보호구역_위도'] = data9['보호구역_위도'].astype(float)
data9['geometry'] = data9.apply(lambda row : Point([row['보호구역_경도'], row['보호구역_위도']]), axis=1)
data9['geometry'] = gpd.GeoDataFrame(data9['geometry'], geometry='geometry')
data9.head()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,geometry
0,특수학교,성심학교,N,0,25,127.017521,37.176762,POINT (127.0175214 37.17676179999999)
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103,POINT (127.0479566 37.1481034)
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197,POINT (127.0397087 37.1841969)
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106,POINT (127.0578885 37.1591063)
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749,POINT (127.0632819 37.162749)


In [6]:
pts = gpd.GeoDataFrame(data9['geometry'], geometry='geometry')
print(pts.crs)

pts.crs = {'init':'epsg:4326'} #좌표계가 지정되지 않아 알맞는(4326)좌표계를 설정해줍니다.
pts = pts.to_crs({'init':'epsg:5181'}) #4326좌표계는 미터단위 분석이 불가능, 100미터 버퍼 생성이 가능한 좌표계로 변환합니다.
print(pts.crs)

buf=pts.buffer(100) #100미터 원 버퍼를 만드는 코드입니다.
print(buf.crs)
buf = buf.to_crs({'init':'epsg:4326'}) #본 분석의 기준에 맞게 좌표계를 원상태로 돌려뒀습니다.
print(buf.crs)

None
+init=epsg:5181 +type=crs
+init=epsg:5181 +type=crs
+init=epsg:4326 +type=crs


In [7]:
data31 = gpd.read_file("31.오산시_법정경계(시군구).geojson") #시각화중 오산시의 경계를 표시하기 위함입니다.

In [8]:
# 시각화
center = [37.165397, 127.048396]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#475974'}
style2 = {'fillColor': '#ED9253', 'color': '#ED9253'}

folium.GeoJson(data=data31["geometry"], style_function=lambda x:style1).add_to(m) #오산시경계
for i in range(len(data9)):
    folium.Circle(
        location=[data9.iloc[i]['보호구역_위도'], data9.iloc[i]['보호구역_경도']],
        color='#6B5149',
        radius=3,
    ).add_to(m)

folium.GeoJson(data=buf, style_function=lambda x:style2).add_to(m) #버퍼

m

In [9]:
#시설명을 포함한 버퍼(구역) 데이터프레임을 만듦니다
s_zone_buf=data9[['시설종류','시설명']]
s_zone_buf['geometry']=buf
s_zone_buf=gpd.GeoDataFrame(s_zone_buf)
s_zone_buf.head()

,시설종류,시설명,geometry
0,특수학교,성심학교,"POLYGON ((127.01865 37.17676, 127.01864 37.176..."
1,초등학교,가수초등학교,"POLYGON ((127.04908 37.14810, 127.04908 37.148..."
2,초등학교,광성초등학교,"POLYGON ((127.04083 37.18420, 127.04083 37.184..."
3,초등학교,대호초등학교,"POLYGON ((127.05901 37.15911, 127.05901 37.159..."
4,초등학교,매홀초등학교,"POLYGON ((127.06441 37.16275, 127.06440 37.162..."


In [10]:
s_zone_buf.to_file('어린이보호구역버퍼.geojson',  driver='GeoJSON', index=False, header=True)

## 어린이보호구역 도로 추출
#### 지적도 데이터에서 도로만 추출하는 파이썬 코드입니다

In [11]:
data34 = gpd.read_file("34.오산시_지적도.geojson")
data34.head(2)

,PNU,JIBUN,geometry
0,4137012100103020039,302-39답,"MULTIPOLYGON (((127.06160 37.14889, 127.06151 ..."
1,4137012100103020005,302-5대,"MULTIPOLYGON (((127.06144 37.14884, 127.06141 ..."


In [12]:
data34['type']=data34['JIBUN'].str[-1] #지번 뒤에 한글이 특성을 의미하므로 따로 떼어 확인합니다.
data34.head(4)

,PNU,JIBUN,geometry,type
0,4137012100103020039,302-39답,"MULTIPOLYGON (((127.06160 37.14889, 127.06151 ...",답
1,4137012100103020005,302-5대,"MULTIPOLYGON (((127.06144 37.14884, 127.06141 ...",대
2,4137012100103000012,300-12도,"MULTIPOLYGON (((127.06095 37.14948, 127.06095 ...",도
3,4137012100102900007,290-7도,"MULTIPOLYGON (((127.06084 37.14945, 127.06093 ...",도


In [13]:
road=data34[data34['type']=='도'] #도로는 "도"로 분리 및 표시되어있으므로 "도"만 추출합니다.
road.head(2)

,PNU,JIBUN,geometry,type
2,4137012100103000012,300-12도,"MULTIPOLYGON (((127.06095 37.14948, 127.06095 ...",도
3,4137012100102900007,290-7도,"MULTIPOLYGON (((127.06084 37.14945, 127.06093 ...",도


In [14]:
road.to_file('지적도_도로.geojson',  driver='GeoJSON', index=False, header=True)

#### QGIS 결과 시각화
- qgis이용, "지적도_도로.geojson" 파일과 "어린이보호구역버퍼.geojson"의 **교차영역**을 추출했습니다.
- 결과물은 "지적도_교차영역도로_100원.geojson"으로 저장하여 불러왔습니다.

In [15]:
inter_road = gpd.read_file("지적도_교차영역도로_100원.geojson")
inter_road.head()

,PNU,JIBUN,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,geometry
0,4137011100101510002,151-2 도,유치원,동탄우리유치원,Y,1,4,127.060252,37.196997,"MULTIPOLYGON (((127.06077 37.19778, 127.06070 ..."
1,4137011100101510002,151-2 도,어린이집,베스트어린이집,Y,1,3,127.061423,37.197877,"MULTIPOLYGON (((127.06091 37.19805, 127.06086 ..."
2,4137011100101510003,151-3 도,유치원,동탄우리유치원,Y,1,4,127.060252,37.196997,"MULTIPOLYGON (((127.06064 37.19770, 127.06064 ..."
3,4137011100101510003,151-3 도,어린이집,베스트어린이집,Y,1,3,127.061423,37.197877,"MULTIPOLYGON (((127.06064 37.19770, 127.06064 ..."
4,4137011100101510004,151-4 도,유치원,동탄우리유치원,Y,1,4,127.060252,37.196997,"MULTIPOLYGON (((127.06055 37.19781, 127.06056 ..."


In [16]:
# 지도에 시각화합니다.
center = [37.165397, 127.048396]
m = folium.Map(location=center, zoom_start=13,tiles='cartodbpositron')
style1 = {'fillColor': False, 'color': '#475974'} #짙은남색
style2 = {'fillColor': '#ED9253', 'color': '#ED9253'} #주황색
style3 = {'fillColor': '#F15F5F', 'color': '#F15F5F'} #붉은색

folium.GeoJson(data=data31["geometry"], style_function=lambda x:style1).add_to(m) #오산시경계
folium.GeoJson(data=buf, style_function=lambda x:style2).add_to(m) #보호구역
folium.GeoJson(data=inter_road['geometry'], style_function=lambda x:style3).add_to(m) #보호구역내 도로

m